## Setup

In [22]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Find a Recent Completed Game

Before we can perform advanced analytics, we need to find a completed game to analyze. We'll search the schedule for games with `gameState == 'OFF'` (finished non-preseason games).

In [23]:
from scrapernhl import scrapeSchedule
from datetime import datetime

# Get recent games from any team
schedule = scrapeSchedule("MTL", "20252026")
completed = schedule[schedule['gameState'] == 'OFF']

if len(completed) > 0:
    game_id = completed.iloc[0]['id']
    game_info = completed.iloc[0]
    print(f"Using game: {game_info['awayTeam.abbrev']} @ {game_info['homeTeam.abbrev']}")
    print(f"Date: {game_info['gameDate']}")
    print(f"Game ID: {game_id}")
else:
    print("No completed games found. Using a known game ID...")
    game_id = 2024020001

Using game: MTL @ TOR
Date: 2025-10-08
Game ID: 2025020004


## 1. Complete Game Data with Metadata

The `scrape_game()` function returns comprehensive game data including play-by-play events and roster information in a tuple format.

In [24]:
from scrapernhl import scrape_game

# Get comprehensive game data
game_tuple = scrape_game(game_id=game_id, include_tuple=True)

pbp = game_tuple.data

print(f"Game: {game_tuple.awayTeam} @ {game_tuple.homeTeam}")
print(f"Total events: {len(pbp)}")

print("\nRosters:")
display(game_tuple.rosters)

Game: MTL @ TOR
Total events: 1887

Rosters:


,teamId,playerId,sweaterNumber,positionCode,headshot,firstName.default,lastName.default,firstName.cs,firstName.de,firstName.es,firstName.fi,firstName.sk,firstName.sv,lastName.cs,lastName.fi,lastName.sk,isHome,fullName
0,10,8475166,91,C,https://assets.nhle.com/mugs/nhl/20252026/TOR/...,John,Tavares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,John Tavares
1,10,8475171,95,D,https://assets.nhle.com/mugs/nhl/20252026/TOR/...,Oliver,Ekman-Larsson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Oliver Ekman-Larsson
2,10,8475690,8,D,https://assets.nhle.com/mugs/nhl/20252026/TOR/...,Chris,Tanev,Christopher,Christopher,Christopher,Christopher,Christopher,Christopher,NaN,NaN,NaN,1,Chris Tanev
3,10,8475714,19,C,https://assets.nhle.com/mugs/nhl/20252026/TOR/...,Calle,Jarnkrok,NaN,NaN,NaN,NaN,NaN,NaN,Järnkrok,Järnkrok,Järnkrok,1,Calle Jarnkrok
4,8,8475848,11,R,https://assets.nhle.com/mugs/nhl/20252026/MTL/...,Brendan,Gallagher,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Brendan Gallagher
5,10,8476853,44,D,https://assets.nhle.com/mugs/nhl/20252026/TOR/...,Morgan,Rielly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Morgan Rielly
6,8,8476875,8,D,https://assets.nhle.com/mugs/nhl/20252026/MTL/...,Mike,Matheson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Mike Matheson
7,10,8476931,22,D,https://assets.nhle.com/mugs/nhl/20252026/TOR/...,Jake,McCabe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Jake McCabe
8,10,8476932,41,G,https://assets.nhle.com/mugs/nhl/20252026/TOR/...,Anthony,Stolarz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Anthony Stolarz
9,8,8476981,17,R,https://assets.nhle.com/mugs/nhl/20252026/MTL/...,Josh,Anderson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Josh Anderson


## 2. Expected Goals (xG) Analysis

Expected Goals (xG) is a metric that estimates the probability of a shot resulting in a goal based on factors like shot location, type, and game situation.

In [25]:
from scrapernhl import prepare_pbp_with_xg

# Engineer xG features and predict xG in one call
pbp_with_xg = prepare_pbp_with_xg(pbp)

# Get shots and goals
shots = pbp_with_xg[pbp_with_xg['Event'].isin(['SHOT', 'GOAL', 'MISS'])].copy()

print(f"Total shot attempts: {len(shots)}")

print("\nShot attempts with xG:")
display(shots[['period', 'timeInPeriod', 'Event', 'eventTeam', 'player1Id', 'xG', 'distanceFromGoal']].head(10))

Total shot attempts: 82

Shot attempts with xG:


,period,timeInPeriod,Event,eventTeam,player1Id,xG,distanceFromGoal
19,1,00:27,SHOT,TOR,8476931,0.06,34.23
40,1,00:54,SHOT,TOR,8476853,0.06,22.67
49,1,01:00,GOAL,TOR,8482259,0.13,18.11
66,1,01:23,SHOT,MTL,8482964,0.02,53.71
105,1,02:16,SHOT,MTL,8483515,0.02,14.04
108,1,02:33,SHOT,MTL,8483515,0.08,16.16
158,1,04:07,SHOT,TOR,8481582,0.02,44.05
199,1,05:22,SHOT,MTL,8476981,0.10,24.00
206,1,05:39,GOAL,MTL,8482775,0.04,38.28
226,1,05:50,SHOT,TOR,8477503,0.03,39.82


In [27]:
# Calculate team xG totals
home_team = game_tuple.homeTeam
away_team = game_tuple.awayTeam

home_shots = shots[shots['eventTeam'] == home_team]
away_shots = shots[shots['eventTeam'] == away_team]

home_xg = home_shots['xG'].sum()
away_xg = away_shots['xG'].sum()

home_goals = len(pbp[(pbp['Event'] == 'GOAL') & (pbp['eventTeam'] == home_team)])
away_goals = len(pbp[(pbp['Event'] == 'GOAL') & (pbp['eventTeam'] == away_team)])

print(f"{away_team} @ {home_team}")
print(f"Score: {away_goals} - {home_goals}")
print(f"xG: {away_xg:.2f} - {home_xg:.2f}")
print(f"xG differential: {home_xg - away_xg:.2f} (positive favors {home_team})")

MTL @ TOR
Score: 2 - 5
xG: 2.93 - 3.50
xG differential: 0.57 (positive favors TOR)


## 3. Time on Ice (TOI) Analysis

Calculate how much time each player spent on the ice at different game strengths (5v5, power play, penalty kill, etc.).

In [28]:
from scrapernhl import toi_by_strength

# Calculate TOI by strength
toi_df = toi_by_strength(pbp)

print("TOI by strength:")
display(toi_df.head(10))

TOI by strength:


,strength,seconds,minutes
2,5v5,3015,50.25
0,4v5,240,4.00
1,5v4,240,4.00
4,6*v5,87,1.45
3,5v6*,18,0.30


In [29]:
from scrapernhl import combo_on_ice_stats_both_teams

# Individual player stats
combo_stats = combo_on_ice_stats_both_teams(
    pbp,
    n_team=1,
    m_opp=0,          # set 0 for "vs ANY"
    min_TOI=0,
    include_goalies=False,
    rates=True,
    player_df=game_tuple.rosters  # DataFrame with ids/teams/positions
)

display(combo_stats[['player1Id', 'player1Name', 'player1Position', 'player1Number', 'team', 'opp', 'strength', 'seconds', 'minutes']])

,player1Id,player1Name,player1Position,player1Number,team,opp,strength,seconds,minutes
0,8481618,Alex Newhook,F,15,MTL,TOR,4v5,128.00,2.13
1,8478851,Alexandre Carrier,D,45,MTL,TOR,4v5,131.00,2.18
2,8482964,Arber Xhekaj,D,72,MTL,TOR,4v5,5.00,0.08
3,8478133,Jake Evans,F,71,MTL,TOR,4v5,113.00,1.88
4,8476981,Josh Anderson,F,17,MTL,TOR,4v5,94.00,1.57
...,...,...,...,...,...,...,...,...,...
120,8476853,Morgan Rielly,D,44,TOR,MTL,6*v5,5.00,0.08
121,8478462,Nicolas Roy,F,55,TOR,MTL,6*v5,43.00,0.72
122,8481122,Simon Benoit,D,2,TOR,MTL,6*v5,36.00,0.60
123,8478904,Steven Lorentz,F,18,TOR,MTL,6*v5,65.00,1.08


## 4. Player Combinations Analysis

Analyze which players play together most often by examining 2-player (defensive pairs) and 3-player (forward lines) combinations.

### Defensive Pairs (2-player combinations)

Find the most common defensive pairings at 5v5.

In [8]:
# Get 2-player combinations (defensive pairs)
combo_stats_2 = combo_on_ice_stats_both_teams(
    pbp,
    n_team=2,
    m_opp=0,          # set 0 for "vs ANY"
    min_TOI=60,
    include_goalies=False,
    rates=True,
    player_df=game_tuple.rosters  # DataFrame with ids/teams/positions
)

top_10_pairs_5v5 = (combo_stats_2
                    .query("team_combo_pos == '2D'")  # Get defensive pairs
                    .query("strength == '5v5'")  # Filter for 5v5
                    .nlargest(10, 'seconds')
                    )[['team_combo', 'team_combo_ids', 'team', 'opp', 'strength', 'seconds', 'minutes']]

print("Most common defensive pairs (5v5):")
display(top_10_pairs_5v5)

Most common defensive pairs (5v5):


,team_combo,team_combo_ids,team,opp,strength,seconds,minutes
168,Chris Tanev / Jake McCabe,"(8475690, 8476931)",TOR,MTL,5v5,1065.00,17.75
89,Mike Matheson / Noah Dobson,"(8476875, 8480865)",MTL,TOR,5v5,1026.00,17.10
206,Morgan Rielly / Brandon Carlo,"(8476853, 8478443)",TOR,MTL,5v5,793.00,13.22
71,Kaiden Guhle / Lane Hutson,"(8482087, 8483457)",MTL,TOR,5v5,698.00,11.63
229,Oliver Ekman-Larsson / Simon Benoit,"(8475171, 8481122)",TOR,MTL,5v5,654.00,10.90
28,Alexandre Carrier / Arber Xhekaj,"(8478851, 8482964)",MTL,TOR,5v5,400.00,6.67
32,Alexandre Carrier / Kaiden Guhle,"(8478851, 8482087)",MTL,TOR,5v5,335.00,5.58
41,Arber Xhekaj / Lane Hutson,"(8482964, 8483457)",MTL,TOR,5v5,200.00,3.33
81,Mike Matheson / Alexandre Carrier,"(8476875, 8478851)",MTL,TOR,5v5,107.00,1.78
226,Oliver Ekman-Larsson / Morgan Rielly,"(8475171, 8476853)",TOR,MTL,5v5,104.00,1.73


### Forward Lines (3-player combinations)

Find the most common forward line combinations at 5v5.

In [9]:
# Get 3-player combinations (forward lines)
combo_stats_3 = combo_on_ice_stats_both_teams(
    pbp,
    n_team=3,
    m_opp=0,          # set 0 for "vs ANY"
    min_TOI=60,
    include_goalies=False,
    rates=True,
    player_df=game_tuple.rosters  # DataFrame with ids/teams/positions
)

top_10_lines_5v5 = (combo_stats_3
                    .query("team_combo_pos == '3F'")  # Get offensive lines
                    .query("strength == '5v5'")  # Filter for 5v5
                    .nlargest(10, 'seconds')
                    )[['team_combo', 'team_combo_ids', 'team', 'opp', 'strength', 'seconds', 'minutes']]

print("Most common offensive lines (5v5):")
display(top_10_lines_5v5)

Most common offensive lines (5v5):


,team_combo,team_combo_ids,team,opp,strength,seconds,minutes
124,Nick Suzuki / Cole Caufield / Juraj Slafkovský,"(8480018, 8481540, 8483515)",MTL,TOR,5v5,822.00,13.70
237,John Tavares / William Nylander / Bobby McMann,"(8475166, 8477939, 8482259)",TOR,MTL,5v5,686.00,11.43
175,Auston Matthews / Matias Maccelli / Matthew Knies,"(8479318, 8481711, 8482720)",TOR,MTL,5v5,632.00,10.53
24,Alex Newhook / Oliver Kapanen / Ivan Demidov,"(8481618, 8482775, 8484984)",MTL,TOR,5v5,589.00,9.82
53,Brendan Gallagher / Kirby Dach / Zachary Bolduc,"(8475848, 8481523, 8482737)",MTL,TOR,5v5,519.00,8.65
83,Josh Anderson / Jake Evans / Patrik Laine,"(8476981, 8478133, 8479339)",MTL,TOR,5v5,513.00,8.55
191,Calle Jarnkrok / Nicolas Roy / Steven Lorentz,"(8475714, 8478462, 8478904)",TOR,MTL,5v5,492.00,8.20
244,Max Domi / Dakota Joshua / Nicholas Robertson,"(8477503, 8478057, 8481582)",TOR,MTL,5v5,475.00,7.92
240,Max Domi / Auston Matthews / Matthew Knies,"(8477503, 8479318, 8482720)",TOR,MTL,5v5,103.00,1.72


## 5. On-Ice Statistics by Player

Calculate advanced metrics for each player including Corsi (shot attempts for/against), expected goals, and actual goals when they're on the ice.

In [10]:
from scrapernhl import on_ice_stats_by_player_strength

# Calculate on-ice stats
player_stats = on_ice_stats_by_player_strength(
    pbp_with_xg,
    include_goalies=False,
    rates=True  # Per-60 rates
)

print("On-ice statistics by player:")
display(player_stats.head(10))

On-ice statistics by player:


,player1Id,player1Name,eventTeam,strength,seconds,minutes,CF,CA,FF,FA,SF,SA,GF,GA,xGF,xGA,PF,PA,CF_per60,CA_per60,FF_per60,FA_per60,SF_per60,SA_per60,xGF_per60,xGA_per60,GF_per60,GA_per60,PF_per60,PA_per60
0,8481618,Alex Newhook,MTL,4v5,128.00,2.13,3.00,0.00,2.00,0.00,2.00,0.00,1.00,0.00,0.10,0.00,0.00,0.00,84.38,0.00,56.25,0.00,56.25,0.00,2.85,0.00,28.12,0.00,0.00,0.00
1,8481618,Alex Newhook,MTL,5v4,12.00,0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,8481618,Alex Newhook,MTL,5v5,775.00,12.92,6.00,15.00,5.00,12.00,2.00,9.00,0.00,0.00,0.12,0.45,0.00,0.00,27.87,69.68,23.23,55.74,9.29,41.81,0.57,2.09,0.00,0.00,0.00,0.00
3,8478851,Alexandre Carrier,MTL,4v5,131.00,2.18,1.00,1.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.03,0.00,0.00,27.48,27.48,0.00,27.48,0.00,27.48,0.00,0.87,0.00,0.00,0.00,0.00
4,8478851,Alexandre Carrier,MTL,5v5,913.00,15.22,14.00,17.00,9.00,11.00,6.00,10.00,0.00,1.00,0.27,0.57,0.00,0.00,55.20,67.03,35.49,43.37,23.66,39.43,1.07,2.26,0.00,3.94,0.00,0.00
5,8482964,Arber Xhekaj,MTL,4v5,5.00,0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,8482964,Arber Xhekaj,MTL,5v5,663.00,11.05,15.00,13.00,9.00,7.00,6.00,6.00,1.00,1.00,0.49,0.43,0.00,0.00,81.45,70.59,48.87,38.01,32.58,32.58,2.65,2.34,5.43,5.43,0.00,0.00
7,8475848,Brendan Gallagher,MTL,5v4,46.00,0.77,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,78.26,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,8475848,Brendan Gallagher,MTL,5v5,746.00,12.43,14.00,15.00,7.00,12.00,7.00,8.00,1.00,2.00,0.77,0.82,1.00,0.00,67.56,72.39,33.78,57.91,33.78,38.61,3.69,3.94,4.83,9.65,4.83,0.00
9,8475848,Brendan Gallagher,MTL,5v6*,15.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,240.00


In [11]:
# Best Corsi For % (min 5 min TOI)
qualified = player_stats[player_stats['minutes'] >= 5].copy()
qualified['CF%'] = qualified['CF'] / (qualified['CF'] + qualified['CA']) * 100

print("Best Corsi For % (5+ min TOI):")
display(qualified.nlargest(10, 'CF%')[['player1Name', 'eventTeam', 'strength', 'minutes', 'CF', 'CA', 'CF%']])

Best Corsi For % (5+ min TOI):


,player1Name,eventTeam,strength,minutes,CF,CA,CF%
23,Juraj Slafkovský,MTL,5v5,15.32,22.00,9.00,70.97
41,Nick Suzuki,MTL,5v5,15.92,21.00,9.00,70.00
103,Morgan Rielly,TOR,5v5,17.38,24.00,11.00,68.57
115,Simon Benoit,TOR,5v5,14.40,12.00,6.00,66.67
12,Cole Caufield,MTL,5v5,16.00,22.00,12.00,64.71
68,Brandon Carlo,TOR,5v5,16.08,19.00,12.00,61.29
79,Dakota Joshua,TOR,5v5,9.60,11.00,8.00,57.89
86,John Tavares,TOR,5v5,13.58,13.00,10.00,56.52
55,Zachary Bolduc,MTL,5v5,10.10,14.00,11.00,56.00
45,Noah Dobson,MTL,5v5,18.35,19.00,15.00,55.88


In [12]:
# Best xG differential
qualified['xGF_diff'] = qualified['xGF'] - qualified['xGA']

print("Best xGF differential (5+ min TOI):")
display(qualified.nlargest(10, 'xGF_diff')[['player1Name', 'eventTeam', 'strength', 'minutes', 'xGF', 'xGA', 'xGF_diff']])

Best xGF differential (5+ min TOI):


,player1Name,eventTeam,strength,minutes,xGF,xGA,xGF_diff
12,Cole Caufield,MTL,5v5,16.00,1.53,0.57,0.97
45,Noah Dobson,MTL,5v5,18.35,1.30,0.52,0.78
23,Juraj Slafkovský,MTL,5v5,15.32,1.27,0.52,0.76
41,Nick Suzuki,MTL,5v5,15.92,1.25,0.52,0.74
103,Morgan Rielly,TOR,5v5,17.38,1.07,0.36,0.70
36,Mike Matheson,MTL,5v5,20.08,1.41,0.85,0.56
122,William Nylander,TOR,5v5,14.50,0.91,0.38,0.53
68,Brandon Carlo,TOR,5v5,16.08,0.90,0.43,0.47
86,John Tavares,TOR,5v5,13.58,0.69,0.26,0.43
115,Simon Benoit,TOR,5v5,14.40,0.46,0.06,0.39


## 6. Team-Level Aggregates

Aggregate statistics to the team level, showing performance at different game strengths (5v5, PP, PK, etc.).

In [13]:
from scrapernhl import team_strength_aggregates

# Calculate team stats by strength
team_stats = team_strength_aggregates(
    pbp_with_xg,
    include_goalies=False,
    rates=True,
    min_TOI=1
)

print("Team statistics by strength:")
display(team_stats[['team', 'minutes', 'CF', 'CA', 'xGF', 'xGA', 'GF', 'GA']].sort_values(by=['minutes'], ascending=False))

Team statistics by strength:


,team,minutes,CF,CA,xGF,xGA,GF,GA
2,MTL,50.25,48.00,48.00,2.31,1.86,1.00,3.00
7,TOR,50.25,48.00,48.00,1.86,2.31,3.00,1.00
0,MTL,4.00,5.00,2.00,0.21,0.09,1.00,0.00
1,MTL,4.00,4.00,3.00,0.21,0.00,0.00,0.00
5,TOR,4.00,2.00,5.00,0.09,0.21,0.00,1.00
6,TOR,4.00,3.00,4.00,0.00,0.21,0.00,0.00
4,MTL,1.45,4.00,3.00,0.21,1.55,0.00,2.00
9,TOR,1.45,3.00,4.00,1.55,0.21,2.00,0.00
3,MTL,0.30,0.00,0.00,0.00,0.00,0.00,0.00
8,TOR,0.30,0.00,0.00,0.00,0.00,0.00,0.00


In [14]:
# 5v5 stats only
stats_5v5 = team_stats[team_stats['strength'] == '5v5'].copy()

print("5v5 Team Stats:")
display(stats_5v5[['team', 'minutes', 'CF', 'CA', 'xGF', 'xGA', 'GF', 'GA']])

5v5 Team Stats:


,team,minutes,CF,CA,xGF,xGA,GF,GA
2,MTL,50.25,48.00,48.00,2.31,1.86,1.00,3.00
7,TOR,50.25,48.00,48.00,1.86,2.31,3.00,1.00


## 7. Multi-Game Season Analysis (Preview)

Demonstrate how to scrape and analyze multiple games to build season-long statistics. This example processes 3 games for demonstration.

In [15]:
# Scrape multiple games (just 3 for demonstration)
print("Scraping multiple games for season analysis...")

game_ids_to_scrape = completed.head(3)['id'].tolist()
all_team_stats = []

for gid in game_ids_to_scrape:
    try:
        print(f"Processing game {gid}...")
        # Use different variable names to avoid overwriting main game data
        multi_game_tuple = scrape_game(gid, include_tuple=True)
        multi_pbp = multi_game_tuple.data
        multi_pbp = prepare_pbp_with_xg(multi_pbp)
        
        stats = team_strength_aggregates(multi_pbp)
        stats['game_id'] = gid
        all_team_stats.append(stats)
    except Exception as e:
        print(f"Error with game {gid}: {e}")

# Combine all games
if all_team_stats:
    season_stats = pd.concat(all_team_stats, ignore_index=True)
    
    # Aggregate by team
    team_summary = season_stats.groupby('team').agg({
        'minutes': 'sum',
        'CF': 'sum',
        'CA': 'sum',
        'xGF': 'sum',
        'xGA': 'sum',
        'GF': 'sum',
        'GA': 'sum'
    }).reset_index()
    
    team_summary['CF%'] = 100 * team_summary['CF'] / (team_summary['CF'] + team_summary['CA'])
    
    print("\nSeason stats across sampled games:")
    display(team_summary)

Scraping multiple games for season analysis...
Processing game 2025020004...
Processing game 2025020010...
Processing game 2025020031...

Season stats across sampled games:


,team,minutes,CF,CA,xGF,xGA,GF,GA,CF%
0,CHI,60.00,52.00,53.00,1.72,3.76,2.00,3.00,49.52
1,DET,60.00,57.00,42.00,2.24,1.61,1.00,5.00,57.58
2,MTL,180.00,156.00,165.00,8.30,7.46,10.00,8.00,48.60
3,TOR,60.00,56.00,61.00,3.50,2.93,5.00,2.00,47.86


## Phase 4: Advanced Analytics

New Phase 4 analytics provide comprehensive statistical analysis and visualization capabilities.

### Shot Metrics and Scoring Chances

Calculate shot distance, angle, and identify high-danger scoring chances.

In [16]:
from scrapernhl import identify_scoring_chances, display_scoring_chances

# Identify high-danger, medium-danger, and low-danger chances
# Note: scoring_chance will be NaN for non-shot events (faceoffs, hits, etc.)
pbp_with_chances = identify_scoring_chances(pbp_with_xg)

# Display scoring chances summary
display_scoring_chances(pbp_with_chances)

         Scoring Chances         
╭──────────┬───────┬────────────╮
│ Category │ Count │ Percentage │
├──────────┼───────┼────────────┤
│ High     │ 22    │ 1.2%       │
│ Medium   │ 60    │ 3.2%       │
│ Low      │ 0     │ 0.0%       │
╰──────────┴───────┴────────────╯

### Advanced Corsi and Fenwick

Calculate possession metrics and display formatted stats.

In [17]:
from scrapernhl import calculate_corsi, calculate_fenwick, display_advanced_stats

# Calculate possession metrics for both teams
# The functions now automatically handle both scrape_game format (Event/eventTeam) 
# and legacy format (typeDescKey/eventOwnerTeam)
home_corsi = calculate_corsi(pbp_with_chances, game_tuple.homeTeam)
home_fenwick = calculate_fenwick(pbp_with_chances, game_tuple.homeTeam)

away_corsi = calculate_corsi(pbp_with_chances, game_tuple.awayTeam)
away_fenwick = calculate_fenwick(pbp_with_chances, game_tuple.awayTeam)

print(f"\n{game_tuple.homeTeam} Corsi:")
print(home_corsi)

print(f"\n{game_tuple.awayTeam} Corsi:")
print(away_corsi)

print(f"\n{game_tuple.homeTeam} Fenwick:")
print(home_fenwick)

print(f"\n{game_tuple.awayTeam} Fenwick:")
print(away_fenwick)

# Display formatted advanced stats for home team
print(f"\n{game_tuple.homeTeam} Advanced Stats:")
display_advanced_stats(home_corsi, home_fenwick, title=f"{game_tuple.homeTeam} Advanced Statistics")


TOR Corsi:
{'corsi_for': 49, 'corsi_against': 68, 'corsi_differential': -19, 'corsi_percentage': 41.88034188034188}

MTL Corsi:
{'corsi_for': 68, 'corsi_against': 49, 'corsi_differential': 19, 'corsi_percentage': 58.119658119658126}

TOR Fenwick:
{'fenwick_for': 35, 'fenwick_against': 47, 'fenwick_differential': -12, 'fenwick_percentage': 42.68292682926829}

MTL Fenwick:
{'fenwick_for': 47, 'fenwick_against': 35, 'fenwick_differential': 12, 'fenwick_percentage': 57.3170731707317}

TOR Advanced Stats:


             TOR Advanced Statistics              
╭─────────┬─────┬─────────┬──────────────┬───────╮
│ Metric  │ For │ Against │ Differential │ %     │
├─────────┼─────┼─────────┼──────────────┼───────┤
│ Corsi   │ 49  │ 68      │ -19          │ 41.9% │
│ Fenwick │ 35  │ 47      │ -12          │ 42.7% │
╰─────────┴─────┴─────────┴──────────────┴───────╯

### Comprehensive Analytics Report

Generate a complete analytics report with all metrics in one call.

In [18]:
from scrapernhl import create_analytics_report, print_analytics_summary

# Generate complete report for home team
# The function now automatically handles scrape_game data format
home_report = create_analytics_report(
    pbp_with_xg,
    shifts_df=None,
    team=game_tuple.homeTeam
)

print(f"\n{game_tuple.homeTeam} Analytics Report:")
print(f"Generated at: {home_report['generated_at']}")

# Print complete summary
print_analytics_summary(home_report)

# Generate report for away team too
away_report = create_analytics_report(
    pbp_with_xg,
    shifts_df=None,
    team=game_tuple.awayTeam
)

print(f"\n{game_tuple.awayTeam} Analytics Report:")
print_analytics_summary(away_report)

ℹ Generating analytics report for TOR...

✓ Analytics report generated


TOR Analytics Report:
Generated at: 2026-01-06T19:19:49.982634


───────────────────────────────────────────── Analytics Report - TOR ──────────────────────────────────────────────

Generated: 2026-01-06T19:19:49.982634

...

       Team Statistics        
╭────────────────────┬───────╮
│ Metric             │ Value │
├────────────────────┼───────┤
│ Goals              │ 2     │
│ Shots              │ 29    │
│ Shot Percentage    │ 6.9   │
│ Hits               │ 27    │
│ Giveaways          │ 13    │
│ Takeaways          │ 3     │
│ Penalties          │ 2     │
│ Faceoff Wins       │ 24    │
│ Faceoff Total      │ 59    │
│ Faceoff Percentage │ 40.7  │
╰────────────────────┴───────╯

               Advanced Statistics                
╭─────────┬─────┬─────────┬──────────────┬───────╮
│ Metric  │ For │ Against │ Differential │ %     │
├─────────┼─────┼─────────┼──────────────┼───────┤
│ Corsi   │ 68  │ 49      │ 19           │ 58.1% │
│ Fenwick │ 47  │ 35      │ 12           │ 57.3% │
╰─────────┴─────┴─────────┴──────────────┴───────╯

      Performance by Score Differential       
╭────────────────┬────────┬────┬────┬────────╮
│ Situation      │ Events │ CF │ CA │ CF%    │
├────────────────┼────────┼────┼────┼────────┤
│ Trailing by 2+ │ 31     │ 4  │ 0  │ 100.0% │
│ Down by 1      │ 287    │ 11 │ 4  │ 73.3%  │
│ Tied           │ 1229   │ 47 │ 35 │ 57.3%  │
│ Up by 1        │ 308    │ 6  │ 9  │ 40.0%  │
│ Leading by 2+  │ 29     │ 0  │ 1  │ 0.0%   │
╰────────────────┴────────┴────┴────┴────────╯

      Shooting Patterns by Distance      
╭──────────┬───────┬───────┬────────────╮
│ Distance │ Shots │ Goals │ Shooting % │
├──────────┼───────┼───────┼────────────┤
│ 0-20ft   │ 19    │ 1     │ 5.3%       │
│ 20-40ft  │ 14    │ 1     │ 7.1%       │
│ 40-60ft  │ 9     │ 0     │ 0.0%       │
│ 60-100ft │ 5     │ 0     │ 0.0%       │
╰──────────┴───────┴───────┴────────────╯

────────────────────────────────────────────────── End of Report ──────────────────────────────────────────────────

,team,generated_at,shooting_patterns,team_stats.goals,team_stats.shots,team_stats.shot_percentage,team_stats.hits,team_stats.giveaways,team_stats.takeaways,team_stats.penalties,team_stats.faceoff_wins,team_stats.faceoff_total,team_stats.faceoff_percentage,team_stats.corsi.corsi_for,team_stats.corsi.corsi_against,team_stats.corsi.corsi_differential,team_stats.corsi.corsi_percentage,team_stats.fenwick.fenwick_for,team_stats.fenwick.fenwick_against,team_stats.fenwick.fenwick_differential,team_stats.fenwick.fenwick_percentage,score_effects.trailing.corsi_for,score_effects.trailing.corsi_against,score_effects.trailing.corsi_differential,score_effects.trailing.corsi_percentage,score_effects.trailing.events,score_effects.down_one.corsi_for,score_effects.down_one.corsi_against,score_effects.down_one.corsi_differential,score_effects.down_one.corsi_percentage,score_effects.down_one.events,score_effects.tied.corsi_for,score_effects.tied.corsi_against,score_effects.tied.corsi_differential,score_effects.tied.corsi_percentage,score_effects.tied.events,score_effects.up_one.corsi_for,score_effects.up_one.corsi_against,score_effects.up_one.corsi_differential,score_effects.up_one.corsi_percentage,score_effects.up_one.events,score_effects.leading.corsi_for,score_effects.leading.corsi_against,score_effects.leading.corsi_differential,score_effects.leading.corsi_percentage,score_effects.leading.events
0,TOR,2026-01-06T19:19:49.982634,"[{'distance_range': '0-20ft', 'total_shots': 1...",5,22,22.73,35,12,2,2,35,59,59.32,49,68,-19,41.88,35,47,-12,42.68,0,4,-4,0.00,31,4,11,-7,26.67,287,35,47,-12,42.68,1229,9,6,3,60.00,308,1,0,1,100.00,29
